# overview

### Preperation | 路径准备


In [4]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import os
import torch
pdbbind_path = "/home/jovyan/dataspace/PDBbind2020/all_pdbbind"
proto_path = "/home/jovyan/dataspace/Prototype"
p2rank = "bash /home/jovyan/TankBind/p2rank_2.3/prank"

In [5]:
tankbind_src_folder_path = "../tankbind_prototype/"
import sys
sys.path.insert(0, tankbind_src_folder_path)

## 数据整理

#### 「折叠」读取表格和 `pdb` 文件。

In [3]:
if False:
    session_table = pd.read_csv("/home/jovyan/dataspace/Liyou/jingsen.csv")

    sample_id = list(range(len(session_table)))
    sample_id = ["ID_"+str(_) for _ in sample_id]
    session_table['sample_id'] = sample_id

In [4]:
if False:
    pdb_in_session = session_table.pdb_id.unique()
    pdb_in_pdbbind = os.listdir(pdbbind_path)
    for _ in tqdm(pdb_in_session):
        if _.lower() not in pdb_in_pdbbind:
            print(_, end=" ")
    # All PDBs in session table could be found in pdbbind2020
    pdb_in_session = list(pdb_in_session)

#### 「折叠」遍历 PDBBind2020 数据库的 ligand 文件，为读取最佳口袋坐标做准备。

In [5]:
if False:
    from feature_utils import read_mol
    from rdkit import RDLogger
    RDLogger.DisableLog('rdApp.*')
    pdb_list = []
    problem_list = []
    for pdb in tqdm(pdb_in_session):
        pdb = pdb.lower()
        sdf_fileName = f"{pdbbind_path}/{pdb}/{pdb}_ligand.sdf"
        mol2_fileName = f"{pdbbind_path}/{pdb}/{pdb}_ligand.mol2"
        mol, problem = read_mol(sdf_fileName, mol2_fileName)
        if problem:
            problem_list.append(pdb)
            continue
        pdb_list.append(pdb)
    print(problem_list)

`problem_list = ['2foy', '2fou', '2fov', '3zp9', '1h07', '3dwb', '1a7x', '1qpf', '4qxs', '2ovz', '1hv5', '1dmt', '3fuc', '4g8l', '1tlp', '4dt6', '3bwf', '3ntp', '2e94', '4lkm', '3mi2', '3gep', '3ggj', '1bzy', '4bxk', '2oc2', '3kck', '3djx', '2pll', '5aqk', '5ipc', '5kly', '5hlm', '4l4l', '4jbs', '5kyk', '3bbb', '4e67', '3fvh', '4lkl', '4r07', '4loi', '3hik', '4e9d', '4e9c', '5km1', '5kma', '4k10']`

针对这些蛋白，将读取文件坐标找到最佳口袋。

#### 「折叠」准备表格
最终结果已导出为 `multiton_table.csv`，故前面的步骤都跳过。

In [7]:
if False:
    session_table['session_au'] = session_table['assay_id'].astype(str) + "_" + session_table['uniprot_id']
    session_count = session_table[['session_au', 'kekule_smiles', 'pdb_id']].groupby('session_au').nunique()
    session_count.reset_index(inplace=True)

    singleton_session = list(session_count[session_count.kekule_smiles==1].session_au)
    multiton_session = list(session_count[session_count.kekule_smiles!=1].session_au)

    ms_dict ={_:"singleton" for _ in singleton_session}
    ms_dict.update({_:"multiton" for _ in multiton_session})

    session_table['session_au_usability'] = session_table.session_au.apply(lambda x: ms_dict[x])
    
    
    from feature_utils import generate_sdf_from_smiles_using_rdkit
    multiton_table = session_table[session_table.session_au_usability=="multiton"]
    multiton_table

    kekule_smiles = list(multiton_table.kekule_smiles.unique())
    len(kekule_smiles)

生成 `smiles_name` 词典，以免直接用 `kekule_smiles` 命名文件导致的问题。

读取 `smiles_name` 词典。

In [8]:
if False:
    import torch
    for i, smiles in enumerate(kekule_smiles):
        smiles_dict[smiles] = f"smiles_{i}"
    import torch
    torch.save(smiles_dict, "/home/jovyan/dataspace/Prototype/dicts/FULL_smiles_dict.pt")
    


根据 `kekule_smiles` 生成小分子文件。只需要执行一次。
这里或许应该生成一份，shifted 的小分子文件！！！

In [9]:
if False:
    error_smiles = []
    for i, smiles in tqdm(enumerate(kekule_smiles), total=len(kekule_smiles)):
        try:
            smiles_name = smiles_dict[smiles]
            generate_sdf_from_smiles_using_rdkit(smiles=smiles, rdkitMolFile=f"/home/jovyan/dataspace/Prototype/rdkit_generated_sdfs_shifted/{smiles_name}.sdf", shift_dis=0)
        except:
            error_smiles.append((smiles, smiles_dict[smiles]))

清理错误的样本，并保存最后的 `multiton_table`。过程只需要执行一次。
```
error_smiles = ["smiles_12096",  "smiles_12097", "smiles_12098", "smiles_37527"]
```

In [12]:
if False:
    smiles_files = os.listdir("/home/jovyan/dataspace/Prototype/rdkit_generated_sdfs")
    smiles_files = [_.replace(".sdf", "") for _ in smiles_files]

    error_smiles = []
    for _ in tqdm(kekule_smiles):
        if smiles_dict[_] not in smiles_files:
            error_smiles.append((_, smiles_dict[_]))

    real_error_smiles = []
    print("LENGTH", len(error_smiles))
    a = 0
    for i, error in tqdm(enumerate(error_smiles), total=len(error_smiles)):
        smiles, smiles_name = error
        try:
            generate_sdf_from_smiles_using_rdkit(smiles=smiles, rdkitMolFile=f"/home/jovyan/dataspace/Prototype/rdkit_generated_sdfs/{smiles_name}.sdf", shift_dis=0)
        except:
            a += 1
            print(smiles_name, end=" ")
            real_error_smiles.append((smiles, smiles_dict[smiles]))
    print("\nLENGTH", a)


    all_smiles_file = os.listdir("/home/jovyan/dataspace/Prototype/rdkit_generated_sdfs")
    all_smiles_file = [_.replace(".sdf", "") for _ in all_smiles_file]


    for _ in tqdm(kekule_smiles):
        if smiles_dict[_] not in all_smiles_file:
            print(smiles_dict[_])


    multiton_table['kekule_smiles_name'] = multiton_table.kekule_smiles.apply(lambda x: smiles_dict[x])


    error_smiles = ["smiles_12096",  "smiles_12097", "smiles_12098", "smiles_37527"]


    multiton_table = multiton_table[~multiton_table.kekule_smiles_name.isin(error_smiles)]
    multiton_table.to_csv("/home/jovyan/dataspace/Prototype/tables/FULLRAW_multiton_table_with_smiles_name.csv")

/tmp/ipykernel_89335/2615119571.py:42: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  multiton_table = pd.read_csv("/home/jovyan/dataspace/Prototype/tables/multiton_table_with_smiles_name.csv", index_col=0)


#### 「折叠」读取 `smiles_dict.pt`，读取 `multiton_table.csv`。

In [6]:
if False:
    smiles_dict = torch.load("/home/jovyan/dataspace/Prototype/smiles_dict.pt")
    multiton_table = pd.read_csv("/home/jovyan/dataspace/Prototype/tables/FULLRAW_multiton_table_with_smiles_name.csv", index_col=0)

/tmp/ipykernel_90545/2114790264.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  multiton_table = pd.read_csv("/home/jovyan/dataspace/Prototype/tables/multiton_table_with_smiles_name.csv", index_col=0)


### 「折叠」<font color='red'>OUT OF DATE</font>  为 `RMSD Loss` 重写小分子

for ease of RMSD evaluation later, we renumber the atom index to be consistent with the smiles
因为没有 RMSD loss， 可能不需要这一项，暂时跳过

```
from feature_utils import write_renumbered_sdf
import os
```

```
toFolder = f"{pre}/renumber_atom_index_same_as_smiles"
os.system(f"mkdir -p {toFolder}")
for pdb in tqdm(pdb_list):
    sdf_fileName = f"{pre}/{pdb}/{pdb}_ligand.sdf"
    mol2_fileName = f"{pre}/{pdb}/{pdb}_ligand.mol2"
    toFile = f"{toFolder}/{pdb}.sdf"
    write_renumbered_sdf(toFile, sdf_fileName, mol2_fileName)
```

### 「折叠」<font color='red'>OUT OF DATE</font> `P2RANK` 的 `pdb` 预处理


process PDBbind proteins, removing extra chains, cutoff 10A
这些步骤被跳过，我们会直接使用 pdb 文件切出的口袋。

```
toFolder = f"{pre}/protein_remove_extra_chains_10A/"
os.system(f"mkdir -p {toFolder}")

```
input_ = []
cutoff = 10
for pdb in data.pdb.values:
    pdbFile = f"{pre}/{pdb}/{pdb}_protein.pdb"
    ligandFile = f"{pre}/renumber_atom_index_same_as_smiles/{pdb}.sdf"
    toFile = f"{toFolder}/{pdb}_protein.pdb"
    x = (pdbFile, ligandFile, cutoff, toFile)
    input_.append(x)

```
from feature_utils import select_chain_within_cutoff_to_ligand_v2

```
import mlcrate as mlc
import os
pool = mlc.SuperPool(64)
pool.pool.restart()
_ = pool.map(select_chain_within_cutoff_to_ligand_v2,input_)
pool.exit()


##### previously, I found that 2r1w has no chain near the ligand.
```
data = data.query("pdb != '2r1w'").reset_index(drop=True)

### 「折叠」蛋白质：口袋切分与特征构建

#### 「折叠」使用 `p2rank` 生成口袋。

```
p2rank_prediction_folder = f"{pre}/p2rank_protein_remove_extra_chains_10A"
os.system(f"mkdir -p {p2rank_prediction_folder}")
ds = f"{p2rank_prediction_folder}/protein_list.ds"
with open(ds, "w") as out:
    for pdb in data.pdb.values:
        out.write(f"../protein_remove_extra_chains_10A/{pdb}_protein.pdb\n")

#### 「折叠」检查已有的 `p2rank` 结果。

根据 `multiton_table.csv` 得到当前的 `pdb_list`。

检查是否所有的 `pdb` 口袋都能在 `p2rank` 输出文件夹中找到。

In [8]:
if False:
    pdb_list = [_.lower() for _ in list(multiton_table.pdb_id.unique())]
    p2rank_prediction_folder = "/home/jovyan/dataspace/Prototype/p2rank_results"


    _ = os.listdir(p2rank_prediction_folder)
    pdb_without_p2rank_result = []
    for pdb in tqdm(pdb_list):
        if f"{pdb.lower()}_protein.pdb_predictions.csv" in _:
            continue
        else:
            pdb_without_p2rank_result.append(pdb)
            print(pdb, end=", ")
    if pdb_without_p2rank_result == []:
        print("P2RANK YES.")

  0%|          | 0/9092 [00:00<?, ?it/s]

P2RANK YES.


#### 「折叠」准备 `protein_features`。

In [24]:
if False:
    tankbind_data_path = f"/home/jovyan/dataspace/Prototype/tankbind_data"
    os.system(f"mkdir -p {tankbind_data_path}")
    pdb_list = [_.lower() for _ in list(multiton_table.pdb_id.unique())]
    d_list = []

    for name in tqdm(pdb_list):
        p2rankFile = f"{p2rank_prediction_folder}/{name}_protein.pdb_predictions.csv"
        d = pd.read_csv(p2rankFile)
        d.columns = d.columns.str.strip()
        d_list.append(d.assign(name=name))
    d = pd.concat(d_list).reset_index(drop=True)
    d.reset_index(drop=True).to_feather(f"{tankbind_data_path}/p2rank_result.feather")

    tankbind_data_path = f"/home/jovyan/dataspace/Prototype/tankbind_data"
    d = pd.read_feather(f"{tankbind_data_path}/p2rank_result.feather")

    pockets_dict = {}
    for name in tqdm(pdb_list):
        pockets_dict[name] = d[d.name == name].reset_index(drop=True)
        
    torch.save(pockets_dict, "/home/jovyan/dataspace/Prototype/dicts/FULL_pockets_dict.pt")

  0%|          | 0/9092 [00:00<?, ?it/s]

In [75]:
if False:
    
    from feature_utils import get_protein_feature

    input_ = []
    protein_embedding_folder = f"{tankbind_data_path}/gvp_protein_embedding"
    os.system(f"mkdir -p {protein_embedding_folder}")
    for pdb in name_list:
        proteinFile = f"{pdbbind_path}/{pdb}/{pdb}_protein.pdb"
        toFile = f"{protein_embedding_folder}/{pdb}.pt"
        x = (pdb, proteinFile, toFile)
        input_.append(x)
        

    from Bio.PDB import PDBParser
    from feature_utils import get_clean_res_list
    import torch
    torch.set_num_threads(1)

    def batch_run(x):
        protein_dict = {}
        pdb, proteinFile, toFile = x
        parser = PDBParser(QUIET=True)
        s = parser.get_structure(pdb, proteinFile)
        res_list = get_clean_res_list(s.get_residues(), verbose=False, ensure_ca_exist=True)
        protein_dict[pdb] = get_protein_feature(res_list)
        torch.save(protein_dict, toFile)
        

    import mlcrate as mlc
    import os
    pool = mlc.SuperPool(64)
    pool.pool.restart()
    _ = pool.map(batch_run,input_)
    pool.exit()


    import torch
    protein_dict = {}
    for pdb in tqdm(name_list):
        protein_dict.update(torch.load(f"{protein_embedding_folder}/{pdb}.pt"))
        
        
    torch.save(protein_dict, "/home/jovyan/dataspace/Prototype/dicts/FULL_protein_dict.pt")




#### 「折叠」读取已保存的 `protein_dict` 和 `pockets_dict`

In [10]:
if False:
    tankbind_data_path = f"/home/jovyan/dataspace/Prototype/tankbind_data"
    protein_dict = torch.load("/home/jovyan/dataspace/Prototype/dicts/FULL_protein_dict.pt")
    pockets_dict = torch.load("/home/jovyan/dataspace/Prototype/dicts/FULL_pockets_dict.pt")

### 「折叠」小分子：特征构建

In [11]:
if False:
    from feature_utils import extract_torchdrug_feature_from_mol
    from feature_utils import read_mol

    compound_dict = {}
    skip_smiles_list = []
    for smiles_name in tqdm(multiton_table.kekule_smiles_name.unique()):
        mol, _ = read_mol(f"/home/jovyan/dataspace/Prototype/rdkit_generated_sdfs/{smiles_name}.sdf", None)
        # extract features from sdf.
        try:
            compound_dict[smiles_name] = extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True)  # self-dock set has_LAS_mask to true
        except Exception as e:
            print(smiles_name, e)
            skip_smiles_list.append(smiles_name)

  0%|          | 0/113099 [00:00<?, ?it/s]

/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `Na`
  warnings.warn("Unknown value `%s`" % x)
/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `Li`
  warnings.warn("Unknown value `%s`" % x)


smiles_5544 max(): Expected reduction dim to be specified for input.numel() == 0. Specify the reduction dim with the 'dim' argument.


/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `K`
  warnings.warn("Unknown value `%s`" % x)
/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `Ag`
  warnings.warn("Unknown value `%s`" % x)
/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `As`
  warnings.warn("Unknown value `%s`" % x)
/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `Te`
  warnings.warn("Unknown value `%s`" % x)


smiles_39785 max(): Expected reduction dim to be specified for input.numel() == 0. Specify the reduction dim with the 'dim' argument.
smiles_39786 max(): Expected reduction dim to be specified for input.numel() == 0. Specify the reduction dim with the 'dim' argument.
smiles_39787 max(): Expected reduction dim to be specified for input.numel() == 0. Specify the reduction dim with the 'dim' argument.
smiles_39788 max(): Expected reduction dim to be specified for input.numel() == 0. Specify the reduction dim with the 'dim' argument.
smiles_40760 max(): Expected reduction dim to be specified for input.numel() == 0. Specify the reduction dim with the 'dim' argument.
smiles_40762 max(): Expected reduction dim to be specified for input.numel() == 0. Specify the reduction dim with the 'dim' argument.
smiles_40763 max(): Expected reduction dim to be specified for input.numel() == 0. Specify the reduction dim with the 'dim' argument.
smiles_40764 max(): Expected reduction dim to be specified for

```
skip_smiles_list = ['smiles_5544', 'smiles_39785', 'smiles_39786', 'smiles_39787', 'smiles_39788', 'smiles_40760', 'smiles_40762', 'smiles_40763', 'smiles_40764', 'smiles_40766', 'smiles_43789']

In [12]:
if False:
    torch.save(compound_dict, f"/home/jovyan/dataspace/Prototype/dicts/FULL_compound_torchdrug_features.pt")

In [14]:
if False:
    multiton_table = multiton_table[~multiton_table.kekule_smiles_name.isin(skip_smiles_list)]
    multiton_table.to_csv("/home/jovyan/dataspace/Prototype/tables/FULLRAW_multiton_table_v2.csv")

In [15]:
if False:
    multiton_table = pd.read_csv("/home/jovyan/dataspace/Prototype/tables/FULLRAW_multiton_table_v2.csv", index_col=0)
    multiton_table['session_aus'] = multiton_table['session_au'] + "_" + multiton_table['kekule_smiles']
    data = multiton_table.copy()

/tmp/ipykernel_90545/3151936896.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  multiton_table = pd.read_csv("/home/jovyan/dataspace/Prototype/tables/multiton_table_v2.csv", index_col=0)


### 「折叠」表格处理

#### 「折叠」导入分组文件以进行 `train-ood-iid-val` 分组

In [66]:
if False:
    drugood_split = pd.read_csv("/home/jovyan/dataspace/Prototype/tables/non_drugood_split2.csv")
    drugood_split=drugood_split.fillna("-")
    drugood_split.split_tag = drugood_split.split_tag.apply(lambda x: "train" if x=="-" else x)
    drugood_split["session_aus"] = drugood_split['assay_id'].astype(str) + "_" + drugood_split['uniprot_id'] + "_" + drugood_split['kekule_smiles']
    drugood_split=drugood_split[['session_aus', 'split_tag']]
    
    data = pd.merge(left=multiton_table, right=drugood_split, how="left", on="session_aus")
    data.to_csv("/home/jovyan/dataspace/Prototype/tables/FULLRAW_info.csv")

#### 「折叠」读取 `raw_info.csv` 为 `data`

In [94]:
if False:
    data = pd.read_csv("/home/jovyan/dataspace/Prototype/tables/FULLRAW_info.csv", index_col=0)

#### 「折叠」处理`data`生成 `info` 并保存

In [128]:
if False:
    info = []
    err_pdb_list=[]
    for i, line in tqdm(data.iterrows(), total=data.shape[0]):
        
        sample_id = line['sample_id']
        pdb_id = line['pdb_id']
        uniprot_id = line['uniprot_id']
        assay_id = line['assay_id']
        smiles_name = line['kekule_smiles_name']
        smiles = line['kekule_smiles']
        session_au = line['session_au']
        session_aus = line['session_aus']
        
        value = line['value']
        docking_score = line['docking_score']
        mmgbsa_binding_energy = line['mmgbsa_binding_energy']
        qed = line["qed"]
        
        split_tag = line['split_tag']
        
        protein_name = pdb_id.lower()
        try:
            pocket = pockets_dict[protein_name].head(20)
        except:
            err_pdb_list.append(pdb)
            continue
        pocket.columns = pocket.columns.str.strip()
        pocket_coms = pocket[['center_x', 'center_y', 'center_z']].values

        # protein center as a block.
        protein_com = protein_dict[protein_name][0].numpy().mean(axis=0).astype(float).reshape(1, 3)
        info.append([sample_id, pdb_id, uniprot_id, assay_id, smiles_name, smiles, session_au, session_aus, protein_com, protein_name+"_c", 
                     value, docking_score, mmgbsa_binding_energy, qed, split_tag])
        
        for idx, pocket_line in pocket.iterrows():
            pdb_idx = f"{protein_name}_{idx}"
            info.append([sample_id, pdb_id, uniprot_id, assay_id, smiles_name, smiles, session_au, session_aus, pocket_coms[idx].reshape(1, 3), pdb_idx, 
                         value, docking_score, mmgbsa_binding_energy, qed, split_tag])
            
    info = pd.DataFrame(
        info, 
        columns=['sample_id', 'pdb_id', 'uniprot_id', 'assay_id', 'smiles_name', 'kekule_smiles', 'session_au', 'session_aus', 'pocket_com', 'pocket_name', 
                 'value', 'docking_score','mmgbsa_binding_energy', 'qed', 'split_tag'])
    print(len(info))


    len(err_pdb_list)
    # 0


    info.to_csv("/home/jovyan/dataspace/Prototype/tables/FULLRAW_info_all_pockets.csv")

  0%|          | 0/695316 [00:00<?, ?it/s]

5886176


#### 「折叠」读取 `info_all_pockets` 为 `info`

In [74]:
if False:
    info = pd.read_csv("/home/jovyan/dataspace/Prototype/tables/FULLRAW_info_all_pockets.csv", index_col=0)

0

#### 「折叠」使用此前生成的 `ligand_coor_dict` 来指定口袋，生成 `pdb_id_to_pocket_name_dict`

In [101]:
if False:
    
    ligand_coor_dict = torch.load("/home/jovyan/dataspace/Prototype/dicts/ARC_ligand_coor_dict.pt")
    
    
    pdb_id_to_pocket_name_dict ={}
    error_pdb_id_list = []
    for PDB_id in tqdm(ligand_coor_dict, total=len(ligand_coor_dict)):
        try:
            pdb_id = PDB_id.lower()
            ligand_center = ligand_coor_dict[PDB_id][0]
            p2rankFile = f"/home/jovyan/dataspace/Prototype/p2rank_results/{pdb_id}_protein.pdb_predictions.csv"
            pocket = pd.read_csv(p2rankFile)
            pocket.columns = pocket.columns.str.strip()
            pocket_coms = pocket[['center_x', 'center_y', 'center_z']].values
            pocket_coms_with_center = np.concatenate([pocket_coms.mean(axis=0).reshape(1,3), pocket_coms])
            pocket_names = [pdb_id + "_c"] + [pdb_id+"_"+str(i) for i in range(len(pocket_coms))]
            right_ith = ((pocket_coms_with_center-ligand_center)**2).sum(axis=1).argmin()
            right_name = pocket_names[right_ith]
            pdb_id_to_pocket_name_dict[PDB_id] = right_name
        except:
            print(PDB_id, end = " ")
            error_pdb_id_list.append(PDB_id)
            pdb_id_to_pocket_name_dict[PDB_id] = "ERROR"
            
    torch.save(pdb_id_to_pocket_name_dict, "/home/jovyan/dataspace/Prototype/dicts/FULL_pdb_id_to_pocket_name_dict.pt")
    torch.save(error_pdb_id_list, "/home/jovyan/dataspace/Prototype/dicts/FULL_pdb_id_to_pocket_name_error_list.pt")


#### 「折叠」读取`pdb_id_to_pocket_name_dict`，并结合 `info_all_pockets` 构造出仅保留单个 `pocket` 的 `info_core_pocket`。
错误的蛋白因为已标注为 `ERROR`，会因为没有名为 `ERROR` 的口袋而被略去。

In [129]:
if False:
    pdb_id_to_pocket_name_dict = torch.load("/home/jovyan/dataspace/Prototype/pdb_id_to_pocket_name_dict.pt")
    info['p2rank_core_pocket'] = info.apply(lambda x: pdb_id_to_pocket_name_dict[x.pdb_id] == x.pocket_name, axis=1)
    
    info.to_csv("/home/jovyan/dataspace/Prototype/tables/FULLRAW_info_all_pockets_noted.csv")
    
    info = info[info.p2rank_core_pocket==True] 
    info.to_csv("/home/jovyan/dataspace/Prototype/tables/FULLRAW_info_core_pocket.csv")

#### 「折叠」读取 `info_core_pocket`，该表格被用于构建最终数据集。

In [ ]:
if False:
    info = pd.read_csv("/home/jovyan/dataspace/Prototype/tables/info_core_pocket.csv", index_col=0)

针对 `val / test` 组的数据，额外提供一个根据 docking_score 预先指定口袋的 info_val_test_best_docking_score

这一步在后续构建两个数据集时也进行了，所以可以不管这一格

In [134]:
if False:
    from pandas import DataFrame
    info_val_test = info[info.split_tag!="train"]
    info_val_test = info_val_test.sort_values('docking_score').groupby('session_aus').apply(DataFrame.head, n=1).reset_index(drop=True)
    info_val_test.to_csv("/home/jovyan/dataspace/Prototype/tables/info_extra_for_val_and_test.csv")

针对 info 和 info_val_test 进行额外的 session_au_usability_check

In [156]:
if False:
    info_count = info[['session_au', 'kekule_smiles', 'pdb_id']].groupby('session_au').nunique()
    info_count.reset_index(inplace=True)
    info_singleton = list(info_count[info_count.kekule_smiles==1].session_au)
    info_multiton = list(info_count[info_count.kekule_smiles!=1].session_au)
    ms_dict ={_:"singleton" for _ in info_singleton}
    ms_dict.update({_:"multiton" for _ in info_multiton})
    info['session_au_usability'] = info.session_au.apply(lambda x: ms_dict[x])
    info = info[info.session_au_usability=="multiton"]
    info.to_csv("/home/jovyan/dataspace/Prototype/tables/FULL_info_core_pocket_multiton.csv")
    info.to_pickle("/home/jovyan/dataspace/Prototype/tables/FULL_info_core_pocket_multiton.pkl")

In [158]:
if False:
    info_count = info_val_test[['session_au', 'kekule_smiles', 'pdb_id']].groupby('session_au').nunique()
    info_count.reset_index(inplace=True)
    info_singleton = list(info_count[info_count.kekule_smiles==1].session_au)
    info_multiton = list(info_count[info_count.kekule_smiles!=1].session_au)
    ms_dict ={_:"singleton" for _ in info_singleton}
    ms_dict.update({_:"multiton" for _ in info_multiton})
    info_val_test['session_au_usability'] = info_val_test.session_au.apply(lambda x: ms_dict[x])
    info_val_test = info_val_test[info_val_test.session_au_usability=="multiton"]
    info_val_test.to_csv("/home/jovyan/dataspace/Prototype/tables/info_extra_for_val_and_test_multiton.csv")

### <font color='gold'>「从这里开始」</font>数据集构建

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import os
import torch
pdbbind_path = "/home/jovyan/dataspace/PDBbind2020/all_pdbbind"
proto_path = "/home/jovyan/dataspace/Prototype"
p2rank = "bash /home/jovyan/TankBind/p2rank_2.3/prank"
tankbind_src_folder_path = "../tankbind_prototype/"
import sys
sys.path.insert(0, tankbind_src_folder_path)

In [3]:
tankbind_data_path = f"/home/jovyan/dataspace/Prototype/tankbind_data"
protein_dict = torch.load("/home/jovyan/dataspace/Prototype/dicts/FULL_protein_dict.pt")
compound_dict = torch.load("/home/jovyan/dataspace/Prototype/dicts/FULL_compound_torchdrug_features.pt")

读取 `info` 并处理 `pocket_com` 列：该列在导出 `csv` 时会变成奇怪的字符串。

In [8]:
info = pd.read_pickle("/home/jovyan/dataspace/Prototype/tables/FULL_info_core_pocket_multiton.pkl")

In [5]:
from data_prototype import TankBindDataSet_prototype

toFileFull = f"/home/jovyan/main_tankbind/dataset_prototype/full/dataset"
os.system(f"rm -rf {toFileFull}")
os.system(f"mkdir -p {toFileFull}")
dataset = TankBindDataSet_prototype(toFileFull, data=info, protein_dict=protein_dict, compound_dict=compound_dict)

Processing...
Done!


['/home/jovyan/main_tankbind/dataset_prototype/full/dataset/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/full/dataset/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/full/dataset/processed/compound.pt']


In [ ]:
t = []
t_dict = {}
data = dataset.data

for i, line in tqdm(data.iterrows(), total=data.shape[0]):
    d = dataset[i]
    sample_id = line['sample_id']
    p_length = d['node_xyz'].shape[0]
    c_length = d['coords'].shape[0]
    y_length = d['y'].shape[0]
    t.append([i, sample_id, p_length, c_length, y_length])
    t_dict[sample_id] = [i, sample_id, p_length, c_length, y_length]
    
torch.save(t, "/home/jovyan/dataspace/Prototype/tables/FULL_supplementary.pt")
torch.save(t_dict, "/home/jovyan/dataspace/Prototype/dicts/FULL_supplementary_dict.pt")

t = pd.DataFrame(t, columns=['index', 'sample_id', 'p_length', 'c_length', 'y_length'])
t.to_csv("/home/jovyan/dataspace/Prototype/tables/FULL_supplementary.csv")

data = pd.concat([data, t[['p_length', 'c_length', 'y_length']]], axis=1)
torch.save(data, f"{toFileFull}/processed/data.pt")

  0%|          | 0/691468 [00:00<?, ?it/s]

In [7]:
data = dataset.data
t = pd.read_csv("/home/jovyan/dataspace/Prototype/tables/FULL_t.csv", index_col=0)
data =pd.concat([data, t[['p_length', 'c_length', 'y_length']]], axis=1)
torch.save(data, f"{toFileFull}/processed/data.pt")

### 验证集和测试集

In [4]:
info = pd.read_csv("/home/jovyan/dataspace/Prototype/tables/FULL_info_core_pocket_multiton.csv", index_col=0)
info.reset_index(inplace=True, drop=True)
info.pocket_com = info.pocket_com.apply(lambda x: np.array([float(_) for _ in x.replace("[[", "").replace("]]", "").split()]))
info2 = info[info.split_tag.isin(["test", 'iid_val', 'ood_val'])].reset_index(drop=True)

In [6]:
info3 = info2.sort_values(by='docking_score', ascending=True).groupby('session_aus', as_index=False).first()

In [7]:
info3

,session_aus,sample_id,pdb_id,uniprot_id,assay_id,smiles_name,kekule_smiles,session_au,pocket_com,pocket_name,value,docking_score,mmgbsa_binding_energy,qed,split_tag,p2rank_core_pocket,session_au_usability
0,101008_Q13093_CCN(CC)CCN(CC1=CC=C(C2=CC=C(C(F)...,ID_40414,5LP1,Q13093,101008,smiles_5980,CCN(CC)CCN(CC1=CC=C(C2=CC=C(C(F)(F)F)C=C2)C=C1...,101008_Q13093,"[28.8894, -27.5758, 4.0971]",5lp1_0,38.00,-10.430974,-77.290802,0.091834,iid_val,True,multiton
1,101008_Q13093_CCN(CC)CCN(CC1=CC=C(C2=CC=C(C(F)...,ID_40428,5LP1,Q13093,101008,smiles_5987,CCN(CC)CCN(CC1=CC=C(C2=CC=C(C(F)(F)F)C=C2)C=C1...,101008_Q13093,"[28.8894, -27.5758, 4.0971]",5lp1_0,0.80,-11.782041,-87.809654,0.078839,iid_val,True,multiton
2,101009_Q13093_CC1=CC=C(C2=CC=C(CN(C)C(=O)CN3C=...,ID_40438,5I9I,Q13093,101009,smiles_5993,CC1=CC=C(C2=CC=C(CN(C)C(=O)CN3C=C(CC4=CN(C)N=C...,101009_Q13093,"[202.0249, 27.3233, 114.6355]",5i9i_0,0.90,-8.839718,-103.795174,0.157879,iid_val,True,multiton
3,101009_Q13093_COC1=CC=C(C2=CC=C(CN(C)C(=O)CN3C...,ID_40457,5I9I,Q13093,101009,smiles_6008,COC1=CC=C(C2=CC=C(CN(C)C(=O)CN3C=C(CC4=CN(C)N=...,101009_Q13093,"[202.0249, 27.3233, 114.6355]",5i9i_0,17.00,-9.582510,-103.853333,0.151225,iid_val,True,multiton
4,101941_P08254_COC(=O)CN1CCCC[C@H](NC(=O)[C@H](...,ID_40519,1BIW,P08254,101941,smiles_6046,COC(=O)CN1CCCC[C@H](NC(=O)[C@H](CC(C)C)[C@@H](...,101941_P08254,"[-3.0935, 50.0764, 50.2571]",1biw_1,215.00,-6.442581,-33.285923,0.316662,iid_val,True,multiton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32626,992786_Q96KQ7_COC1=CC2=C(NC3CCN(C(C)C)CC3)N=C(...,ID_378564,3RJW,Q96KQ7,992786,smiles_55762,COC1=CC2=C(NC3CCN(C(C)C)CC3)N=C(N3CCC(F)(F)CC3...,992786_Q96KQ7,"[-8.5015, 5.081, 47.7921]",3rjw_4,110.00,-10.620724,-94.102745,0.416893,iid_val,True,multiton
32627,992787_Q96KQ7_COC1=CC2=C(NC3CCN(C(C)C)CC3)N=C(...,ID_378587,3RJW,Q96KQ7,992787,smiles_55768,COC1=CC2=C(NC3CCN(C(C)C)CC3)N=C(C3CCOCC3)N=C2C...,992787_Q96KQ7,"[-8.5015, 5.081, 47.7921]",3rjw_4,50000.00,-10.248228,-87.341965,0.391258,iid_val,True,multiton
32628,99805_P03372_CC1=C(C2=CC=C(O)C=C2)N(CC2=CC=C(O...,ID_40352,2IOG,P03372,99805,smiles_4812,CC1=C(C2=CC=C(O)C=C2)N(CC2=CC=C(OCCN3CCCCC3)C=...,99805_P03372,"[32.2618, -0.7075, 26.0667]",2iog_1,1.50,-11.313963,-82.996925,0.361286,ood_val,True,multiton
32629,99805_P03372_CC1=C(C2=CC=C(O)C=C2)N(CC2=CC=C(O...,ID_40353,2IOG,P03372,99805,smiles_4813,CC1=C(C2=CC=C(O)C=C2)N(CC2=CC=C(OCCN3CCCCCC3)C...,99805_P03372,"[32.2618, -0.7075, 26.0667]",2iog_1,3.70,-11.228383,-76.889290,0.331311,ood_val,True,multiton


In [8]:
info3.to_pickle("/home/jovyan/dataspace/Prototype/tables/EXTRA_info_sole_pdb.pkl")

In [9]:
small_pdbs = [_.lower() for _ in info2.pdb_id.unique()]
small_smiles = list(info2.smiles_name.unique())

protein_dict2 = {_:protein_dict[_] for _ in small_pdbs}
compound_dict2 = {_:compound_dict[_] for _ in small_smiles}

In [10]:
toFileExtra = f"/home/jovyan/main_tankbind/dataset_prototype/extra_val_test_reduced_0130"
os.system(f"rm -rf {toFileExtra}")
os.system(f"mkdir -p {toFileExtra}")

from data_prototype import TankBindDataSet_prototype
dataset = TankBindDataSet_prototype(toFileExtra, data=info3, protein_dict=protein_dict2, compound_dict=compound_dict2)

Processing...
Done!


['/home/jovyan/main_tankbind/dataset_prototype/extra_val_test_reduced_0130/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/extra_val_test_reduced_0130/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/extra_val_test_reduced_0130/processed/compound.pt']


In [11]:
t = []
t_dict = {}
data = dataset.data

for i, line in tqdm(data.iterrows(), total=data.shape[0]):
    d = dataset[i]
    sample_id = line['sample_id']
    p_length = d['node_xyz'].shape[0]
    c_length = d['coords'].shape[0]
    y_length = d['y'].shape[0]
    t.append([i, sample_id, p_length, c_length, y_length])
    t_dict[sample_id] = [i, sample_id, p_length, c_length, y_length]
    
torch.save(t, "/home/jovyan/dataspace/Prototype/tables/EXTRA_supplementary.pt")
torch.save(t_dict, "/home/jovyan/dataspace/Prototype/dicts/EXTRA_supplementary_dict.pt")

t = pd.DataFrame(t, columns=['index', 'sample_id', 'p_length', 'c_length', 'y_length'])
t.to_csv("/home/jovyan/dataspace/Prototype/tables/EXTRA_supplementary.csv")

data = pd.concat([data, t[['p_length', 'c_length', 'y_length']]], axis=1)
torch.save(data, f"{toFileExtra}/processed/data.pt")

  0%|          | 0/32631 [00:00<?, ?it/s]

t = []
t_dict = {}
data = dataset.data

for i, line in tqdm(data.iterrows(), total=data.shape[0]):
    d = dataset[i]
    sample_id = line['sample_id']
    p_length = d['node_xyz'].shape[0]
    c_length = d['coords'].shape[0]
    y_length = d['y'].shape[0]
    t.append([i, sample_id, p_length, c_length, y_length])
    t_dict[sample_id] = [i, sample_id, p_length, c_length, y_length]
    
torch.save(t, "/home/jovyan/dataspace/Prototype/tables/full_t.pt")
torch.save(t_dict, "/home/jovyan/dataspace/Prototype/tables/full_t_dict.pt")

t = pd.DataFrame(t, columns=['index', 'sample_id', 'p_length', 'c_length', 'y_length'])
t.to_csv("/home/jovyan/dataspace/Prototype/tables/full_t.csv")

data = pd.concat([data, t[['p_length', 'c_length', 'y_length']]], axis=1)
torch.save(data, f"{toFileFull}/processed/data.pt")

torch.save(data, f"{toFileFull}/processed/data.pt")

dataset = TankBindDataSet_prototype(toFileFull)

t = []
data = dataset.data
pre_pdb = None
for i, line in tqdm(data.iterrows(), total=data.shape[0]):
    pdb = line['compound_name']
    d = dataset[i]
    p_length = d['node_xyz'].shape[0]
    c_length = d['coords'].shape[0]
    y_length = d['y'].shape[0]
    num_contact = (d.y > 0).sum()
    t.append([i, pdb, p_length, c_length, y_length, num_contact])



t = pd.DataFrame(t, columns=['index', 'pdb' ,'p_length', 'c_length', 'y_length', 'num_contact'])
t['num_contact'] = t['num_contact'].apply(lambda x: x.item())

data = pd.concat([data, t[['p_length', 'c_length', 'y_length', 'num_contact']]], axis=1)

native_num_contact = data.query("use_compound_com").set_index("protein_name")['num_contact'].to_dict()
data['native_num_contact'] = data.protein_name.map(native_num_contact)
# data['fract_of_native_contact'] = data['num_contact'] / data['native_num_contact']

torch.save(data, f"{toFilePre}/processed/data.pt")

toFilePre

import torch
info = torch.load(f"{toFilePre}/processed/data.pt")
info['group'].unique()

test = info.query("group == 'test'").reset_index(drop=True)
test_pdb_list = info.query("group == 'test'").protein_name.unique()

test = info.query("group == 'test'").reset_index(drop=True)
test_pdb_list = info.query("group == 'test'").protein_name.unique()

subset_protein_dict = {}
for pdb in tqdm(test_pdb_list):
    subset_protein_dict[pdb] = protein_dict[pdb]

subset_compound_dict = {}
for pdb in tqdm(test_pdb_list):
    subset_compound_dict[pdb] = compound_dict[pdb]


toFilePre = f"{pre}/test_dataset"
os.system(f"mkdir -p {toFilePre}")
dataset = TankBindDataSet(toFilePre, data=test, protein_dict=subset_protein_dict, compound_dict=subset_compound_dict)

def canonical_smiles(smiles):
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles))